# Grasslands Monitoring Tool

In [1]:
# load geemap and other packages
import os
import ee
import geemap
from glob import glob
import pandas as pd
import numpy as np
import geopandas as gpd

In [30]:
# Trigger the authentication flow
# need to do it when it is first time to run this notebook
# will be directed for permission confirmation 
# and The authorization workflow will generate a code, 
# which you should paste in the box below.
ee.Authenticate()

Enter verification code:  4/1AUJR-x5VZZDkJwK8ftTgV4DkCheZlGZxu__rNKDsg2og5iHvqIfV5SukJ3o



Successfully saved authorization token.


In [2]:
ee.Initialize()

### pre-processing grassland project area shapefile

In [3]:
# define project area path
pa_path = r'../Grass/PA_Projected.shp'
name, ext = os.path.splitext(os.path.basename(pa_path))
new_pa_path = os.path.join(os.path.dirname(pa_path), f'{name}_wgs84{ext}')
print(new_pa_path)

../Grass\PA_Projected_wgs84.shp


In [4]:
def crs_conversion(project_area_path):
    project_df = gpd.read_file(project_area_path)
    # print(f'original crs:{project_df.crs}')
    
    project_df_wgs84 = project_df.to_crs(epsg=4326)
    # print(f'new crs: {project_df_wgs84.crs}')
    
    centroid = project_df_wgs84.unary_union.centroid
    lon = centroid.x
    utm_zone = int((lon + 180) // 6) + 1
    epsg_number = 32600+utm_zone
    # print(f'new project crs: EPSG:326{utm_zone}')
    project_df_wgs84_p = project_df_wgs84.to_crs(epsg=epsg_number)
    # remove small polygons less than 1 ha
    project_df_wgs84_p['area'] = project_df_wgs84_p.area
    project_df_filter = project_df_wgs84_p[project_df_wgs84_p['area'] > 10000]
    
    return epsg_number, project_df_filter

In [5]:
epsg_number, project_df_filter = crs_conversion(pa_path)
print(epsg_number)
project_df_filter.to_file(new_pa_path)

32613


### upload project area and buffer area into google earth engine

In [6]:
pa = geemap.shp_to_ee(r'../Grass/PA_Projected_S1.shp')

In [7]:
buffer = geemap.shp_to_ee(r'../Grass/Buffer.shp')

In [7]:
# calculate project area and buffer area
pa_area = round(pa.geometry().area(1).getInfo()/10000, 2)
print(f'Project area:{pa_area} ha')

Project area:555303.58 ha


In [9]:
# calculate project area and buffer area
buffer_area = round(buffer.geometry().area(1).getInfo()/10000, 2)
print(f'Buffer area:{buffer_area} ha')

Buffer area:1943191.41 ha


In [8]:
# Display project area and buffer area in google earth engine
Map = geemap.Map()
Map.centerObject(pa, 10)
Map.addLayer(ee.Image().paint(pa, 0, 2), {'palette': 'red'}, 'Project Area')
Map.addLayer(ee.Image().paint(buffer, 0, 2), {'palette': 'green'}, 'Buffer Area')
Map

Map(center=[29.251727924980372, -106.12141254760482], controls=(WidgetControl(options=['position', 'transparen…

### Landcover composition in project area with GLC-FCS30 dataset

In [9]:
annual = ee.ImageCollection("projects/sat-io/open-datasets/GLC-FCS30D/annual").mosaic()

In [12]:
land_2010 = annual.select(f"b{11}")
land_2015 = annual.select(f"b{16}")
land_2016 = annual.select(f"b{17}")
land_2021 = annual.select(f"b{22}")

In [13]:
landcover_counts = land_2010.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                          geometry=pa, scale=30, crs=f'EPSG:{epsg_number}', maxPixels=1e13)

In [20]:
class_counts = landcover_counts.get('b11').getInfo()

In [23]:
df = pd.DataFrame(list(class_counts.items()), columns=['LandCoverID', 'PixelCount'])
df['Percentage (%)'] = (df['PixelCount'] / df['PixelCount'].sum()) * 100

In [24]:
df

,LandCoverID,PixelCount,Percentage (%)
0,10,3.783098e+02,0.006136
1,11,8.580440e+04,1.391679
2,120,3.680124e+06,59.688695
3,121,2.219216e+01,0.000360
4,130,1.125428e+06,18.253553
5,140,3.027508e+04,0.491038
6,150,3.684314e+02,0.005976
7,181,2.784314e-01,0.000005
8,182,1.512851e+03,0.024537
9,183,2.332000e+02,0.003782


In [26]:
# export data to xlsx
with pd.ExcelWriter(r'../Grass/Land_Cover_GLC-FCS_PA.xlsx') as writer:
    for b, land in zip(['b11', 'b16', 'b17', 'b22'], [land_2010, land_2015, land_2016, land_2021]):
        landcover_counts = land.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                             geometry=pa, scale=30, crs=f'EPSG:{epsg_number}', maxPixels=1e13)
        class_counts = landcover_counts.get(b).getInfo()
        df = pd.DataFrame(list(class_counts.items()), columns=['LandCoverID', 'PixelCount'])
        df['Percentage (%)'] = (df['PixelCount'] / df['PixelCount'].sum()) * 100
        df.to_excel(writer, sheet_name=b, index=True)

### Land cover in project area using ESA land cover dataset 2020 

In [10]:
dataset = ee.ImageCollection('ESA/WorldCover/v100').first()

In [9]:
visualization = {
  'bands': ['Map'],
}

Map.addLayer(dataset.clip(pa), visualization, 'Landcover')

In [11]:
map_class_table = {10: 'Tree Cover',
                   20: 'Shrubland',
                   30: 'Grassland',
                   40: 'Cropland',
                   50: 'Built-up',
                   60: 'Bare/sparse vegetation',
                   70: 'Snow and ice',
                   80: 'Permanent water bodies',
                   90: 'Herbaceous wetland',
                   95: 'Mangroves',
                   100: 'Moss and lichen'}

In [24]:
landcover_counts = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                        geometry=pa, scale=10, crs=f'EPSG:{epsg_number}', maxPixels=1e13)

class_counts = landcover_counts.get('Map').getInfo()

df = pd.DataFrame(list(class_counts.items()), columns=['LandCoverID', 'PixelCount'])
df['Percentage (%)'] = (df['PixelCount'] / df['PixelCount'].sum()) * 100
df['LandCoverClass'] = [map_class_table[int(i)] for i in df['LandCoverID']]
print('Land Cover Composition in Project Area:')
print(df)

Land Cover Composition in Project Area:
  LandCoverID    PixelCount  Percentage (%)          LandCoverClass
0          10  1.597753e+06        2.879145              Tree Cover
1          20  3.381700e+07       60.938081               Shrubland
2          30  1.154721e+07       20.808029               Grassland
3          40  6.811478e+04        0.122743                Cropland
4          50  6.754941e+03        0.012172                Built-up
5          60  8.446449e+06       15.220464  Bare/sparse vegetation
6          80  2.451176e+03        0.004417  Permanent water bodies
7          90  8.296482e+03        0.014950      Herbaceous wetland


In [25]:
landcover_counts = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                        geometry=buffer, scale=10, crs=f'EPSG:{epsg_number}', maxPixels=1e9)

class_counts = landcover_counts.get('Map').getInfo()
df = pd.DataFrame(list(class_counts.items()), columns=['LandCoverID', 'PixelCount'])
df['Percentage (%)'] = (df['PixelCount'] / df['PixelCount'].sum()) * 100
df['LandCoverClass'] = [map_class_table[int(i)] for i in df['LandCoverID']]
print('Land Cover Composition in Buffer Area:')
print(df)

Land Cover Composition in Buffer Area:
  LandCoverID    PixelCount  Percentage (%)          LandCoverClass
0          10  1.002508e+07        5.163815              Tree Cover
1          20  1.183005e+08       60.935396               Shrubland
2          30  3.901542e+07       20.096443               Grassland
3          40  2.035650e+06        1.048543                Cropland
4          50  2.201643e+05        0.113404                Built-up
5          60  2.437904e+07       12.557393  Bare/sparse vegetation
6          80  1.050373e+05        0.054104  Permanent water bodies
7          90  5.999409e+04        0.030902      Herbaceous wetland


### Slope distribution within project area uisng DEM 10m dataset

In [27]:
# DEM dataset for slope, elvation
dataset = ee.Image('USGS/3DEP/10m')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

In [47]:
histogram = slope.reduceRegion(reducer=ee.Reducer.fixedHistogram(0, 90, 18), 
                               geometry=pa, scale=10, crs=f'EPSG:{epsg_number}', maxPixels=1e9)
hist_data = histogram.get('slope').getInfo()

df = pd.DataFrame(hist_data, columns=['Slope [v, v+5)', 'Frequency'])
df['Percent (%)'] = (df['Frequency'] / df['Frequency'].sum()) * 100
df['CumulativeP (%)'] = df['Percent (%)'].cumsum()
print(df)
print(f"Areas with Slope > 65 degrees: {round(100-df.loc[df['Slope [v, v+5)'] == 65]['CumulativeP (%)'].item(), 2)} %")

    Slope [v, v+5)     Frequency  Percent (%)  CumulativeP (%)
0                0  1.007226e+07    78.922811        78.922811
1                5  7.412148e+05     5.807910        84.730721
2               10  6.053972e+05     4.743689        89.474410
3               15  5.009842e+05     3.925544        93.399954
4               20  3.782015e+05     2.963460        96.363415
5               25  2.545643e+05     1.994680        98.358095
6               30  1.401816e+05     1.098416        99.456510
7               35  5.117479e+04     0.400988        99.857499
8               40  1.394693e+04     0.109283        99.966782
9               45  3.220298e+03     0.025233        99.992016
10              50  7.148902e+02     0.005602        99.997617
11              55  2.118078e+02     0.001660        99.999277
12              60  7.696863e+01     0.000603        99.999880
13              65  1.331765e+01     0.000104        99.999984
14              70  2.000000e+00     0.000016       100

In [28]:
histogram = slope.reduceRegion(reducer=ee.Reducer.fixedHistogram(0, 90, 18), 
                               geometry=buffer, scale=10, crs=f'EPSG:{epsg_number}', maxPixels=1e9)
hist_data = histogram.get('slope').getInfo()
df = pd.DataFrame(hist_data, columns=['Slope [v, v+5)', 'Frequency'])
df['Percent (%)'] = (df['Frequency'] / df['Frequency'].sum()) * 100
df['CumulativeP (%)'] = df['Percent (%)'].cumsum()
print(df)
print(f"Areas with Slope > 65 degrees: {round(100-df.loc[df['Slope [v, v+5)'] == 65]['CumulativeP (%)'].item(), 2)} %")

    Slope [v, v+5)     Frequency  Percent (%)  CumulativeP (%)
0                0  2.721442e+07    74.566900        74.566900
1                5  2.765536e+06     7.577505        82.144404
2               10  1.943422e+06     5.324931        87.469336
3               15  1.511573e+06     4.141675        91.611010
4               20  1.234558e+06     3.382660        94.993671
5               25  9.437844e+05     2.585948        97.579618
6               30  5.788637e+05     1.586073        99.165692
7               35  2.220885e+05     0.608517        99.774209
8               40  5.942315e+04     0.162818        99.937027
9               45  1.600536e+04     0.043854        99.980881
10              50  4.763125e+03     0.013051        99.993932
11              55  1.659573e+03     0.004547        99.998479
12              60  4.369412e+02     0.001197        99.999677
13              65  1.070000e+02     0.000293        99.999970
14              70  1.100000e+01     0.000030       100

In [29]:
df.to_excel('2887_PA_RS_Index.xlsx')

### Vegetation indices from Landsat monitoring in project area

In [13]:
def annual_veg_indices_fc(geom):
    # Cloud and shadow masking for Landsat Surface Reflectance
    def mask_clouds_landsat_sr(image):
        qa = image.select('QA_PIXEL')
        cloud = (1 << 3)
        shadow = (1 << 4)
        mask = qa.bitwiseAnd(cloud).eq(0).And(
               qa.bitwiseAnd(shadow).eq(0))
        return image.updateMask(mask)

    # Calculate NDVI, EVI, SAVI, GNDVI
    def calc_indices(image):
        red = image.select('SR_B4').multiply(0.0000275).add(-0.2)
        nir = image.select('SR_B5').multiply(0.0000275).add(-0.2)
        blue = image.select('SR_B2').multiply(0.0000275).add(-0.2)

        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        evi = nir.subtract(red).multiply(2.5).divide(
            nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)
        ).rename('EVI')
        savi = nir.subtract(red).multiply(1.5).divide(nir.add(red).add(0.5)).rename('SAVI')
        gndvi = nir.subtract(image.select('SR_B3').multiply(0.0000275).add(-0.2))\
                    .divide(nir.add(image.select('SR_B3').multiply(0.0000275).add(-0.2)))\
                    .rename('GNDVI')

        return image.addBands([ndvi, evi, savi, gndvi])

    # Load and process image collection
    ls = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
          .filter(ee.Filter.lt('CLOUD_COVER', 20))
          .map(mask_clouds_landsat_sr)
          .map(calc_indices))

    # Per-year reducer
    def per_year(year):
        year = ee.Number(year)
        start = ee.Date.fromYMD(year, 7, 1)
        end = ee.Date.fromYMD(year, 8, 31)

        image = (ls.filterDate(start, end)
                   .filterBounds(geom)
                   .select(['NDVI', 'EVI', 'SAVI', 'GNDVI'])
                   .median())

        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=30,
            maxPixels=1e13
        )

        return ee.Feature(None, {
            'year': year,
            'NDVI': ee.Algorithms.If(stats.contains('NDVI'), stats.get('NDVI'), None),
            'EVI': ee.Algorithms.If(stats.contains('EVI'), stats.get('EVI'), None),
            'SAVI': ee.Algorithms.If(stats.contains('SAVI'), stats.get('SAVI'), None),
            'GNDVI': ee.Algorithms.If(stats.contains('GNDVI'), stats.get('GNDVI'), None)
        })

    years = ee.List.sequence(2013, 2024)
    return ee.FeatureCollection(years.map(per_year))

In [14]:
fc = annual_veg_indices_fc(pa)
results = fc.getInfo()
df_pa = pd.DataFrame([f['properties'] for f in results['features']])
print(df_pa)

         EVI     GNDVI      NDVI      SAVI  year
0   0.201797  0.382514  0.297450  0.201409  2013
1   0.156617  0.340433  0.228699  0.158500  2014
2   0.205735  0.393215  0.313437  0.205261  2015
3   0.172579  0.361207  0.260047  0.173682  2016
4   0.176479  0.357114  0.257900  0.176982  2017
5   0.185290  0.372538  0.281785  0.185642  2018
6   0.155169  0.344363  0.235661  0.157168  2019
7   0.168093  0.357245  0.252631  0.169270  2020
8   0.222993  0.400020  0.322580  0.220455  2021
9   0.174551  0.360149  0.258391  0.176031  2022
10  0.152556  0.342218  0.224135  0.155061  2023
11  0.163552  0.351936  0.243115  0.164288  2024


In [21]:
fc = annual_veg_indices_fc(buffer)
results = fc.getInfo()
df_buffer = pd.DataFrame([f['properties'] for f in results['features']])
print(df_buffer)

         EVI     GNDVI      NDVI      SAVI  year
0   0.211313  0.393116  0.317048  0.210431  2013
1   0.181122  0.367741  0.272625  0.181650  2014
2   0.214229  0.404602  0.331667  0.213336  2015
3   0.182868  0.373849  0.278407  0.183738  2016
4   0.192038  0.373385  0.285242  0.191507  2017
5   0.196720  0.387999  0.305871  0.197021  2018
6   0.167901  0.359532  0.258916  0.169257  2019
7   0.176795  0.368427  0.269440  0.177628  2020
8   0.232735  0.412625  0.342766  0.229851  2021
9   0.180239  0.368425  0.271655  0.181441  2022
10  0.161713  0.353224  0.242044  0.163652  2023
11  0.182499  0.372174  0.274771  0.182307  2024


In [22]:
df_buffer.to_excel('2887_PA_RS_Index.xlsx')